# Transform CSV files into MySQL

In [1]:
import mysql.connector

In [2]:
db = mysql.connector.connect(
  host="localhost",
  user="econ",
  password="econ",
  database="trade_db",
)

In [3]:
cursor = db.cursor()

In [4]:
cursor.execute("SHOW Databases;")
for i in cursor:
    print(i)

('information_schema',)
('trade_db',)


In [ ]:
# Create Table in MySQL
# Product Table (ProductCode, Product, ProductClassificationCode, Product Classification)

In [ ]:
# Create Table in MySQL
# Indicator(IndicatorCode, Indicator Category, Indicator)


In [6]:
import pandas as pd
import numpy as np

In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
pd.set_option('.max_colwidth',None)

In [ ]:
df = pd.read_csv(r"merchandise_values_annual_dataset\merchandise_values_annual_dataset.csv",sep=",")

In [7]:
file2= r"services_annual_dataset.csv"

In [ ]:
df2 = pd.read_csv(file2,sep=',',encoding='cp1252') 

# Economies Table

In [25]:
Region = ['Africa','Asia','Commonwealth of Independent States (CIS), including associate and former member States',\
         'Europe','Middle East', 'North America', 'South and Central America and the Caribbean']

In [26]:
GroupEconomic = ['World', 'Africa', 'African, Caribbean and Pacific States (ACP)',
       'Africa, CIS and Middle East', 'Andean Community (ANDEAN)', 'Asia',
       'Asia-Pacific Economic Cooperation (APEC)',
       'Association of Southeast Asian Nations (ASEAN)',
       'Australia and New Zealand', 'BRIC members', 'BRICS members',
       'Caribbean Community (CARICOM)',
       'Central African Economic and Monetary Community (CAEMC)',
       'Central American Common Market (CACM)',
       'Common Market for Eastern and Southern Africa (COMESA)',
       'Commonwealth of Independent States (CIS), including associate and former member States',
       'Euro Area (19)', 'Europe',
       'European Free Trade Association (EFTA)', 'European Union',
       'European Union (28)', 'Four East Asian traders', 'G-20',
       'Gulf Cooperation Council (GCC)', 'LDC exporters of agriculture',
       'LDC exporters of manufactures', 'LDC non-fuel mineral exporters',
       'LDC oil exporters', 'Least-developed countries', 'Middle East',
       'Non-EU south-eastern Europe', 'Non-EU western Europe',
       'North America', 'North American Free Trade Agreement (NAFTA)',
       'Other Africa', 'Other Asia', 'Other CIS', 'Pacific Alliance',
       'Six East Asian traders',
       'South and Central America and the Caribbean',
       'South Asian Association for Regional Cooperation (SAARC)',
       'Southern African Development Community (SADC)',
       'Southern Common Market (MERCOSUR)',
       'Southern Common Market (MERCOSUR) excluding Venezuela, Bolivarian Republic of',
       'West African Economic and Monetary Union (WAEMU)',
       'West African Economic Community (ECOWAS)', 'WTO Members',
       'WTO Observer governments']

In [79]:
# All Economies 
Economies_dic = {}
for i in df[['ReporterCode','Reporter','ReporterISO3A']].drop_duplicates().values:
    try:
        i[0]=int(i[0])
    except:
        pass
    if str(i[0]) not in Economies_dic.keys():
        Economies_dic[str(i[0])]=i[1:]
for i in df[['PartnerCode','Partner','PartnerISO3A']].drop_duplicates().values:
    try:
        i[0]=int(i[0])
    except:
        pass
    if str(i[0]) not in Economies_dic.keys():
        Economies_dic[str(i[0])]=i[1:]
for i in df2[['ReporterCode','Reporter','ReporterISO3A']].drop_duplicates().values:
    try:
        i[0]=int(i[0])
    except:
        pass
    if str(i[0]) not in Economies_dic.keys():
        Economies_dic[str(i[0])]=i[1:]
for i in df2[['PartnerCode','Partner','PartnerISO3A']].drop_duplicates().values:
    try:
        i[0]=int(i[0])
    except:
        pass
    if str(i[0]) not in Economies_dic.keys():
        Economies_dic[str(i[0])]=i[1:]

In [ ]:
Economies_dic

In [17]:
# Create Table in MySQL
# Economies Table(Code, Name, ISO3A,Is_Reporter, Is_Partner, Is_Group, Is_Region)
cursor.execute('DROP TABLE IF EXISTS economies;')
cursor.execute("create TABLE economies (\
                ecode  VarChar(5) UNIQUE NOT NULL,\
                ename VarChar(100) UNIQUE NOT NULL,\
                iso3a VarChar(6),\
                is_reporter boolean NOT NULL,\
                is_partner boolean NOT NULL,\
                is_region boolean Not NULL,\
                is_group boolean Not NULL,\
                PRIMARY KEY (ecode));")

In [81]:
# insert data:
val = []
for i in Economies_dic:
    iso3a = None if str(Economies_dic[i][1]) =='nan' else Economies_dic[i][1]
    name = Economies_dic[i][0]
    is_region = 1 if name in Region else 0
    is_group = 1 if name in GroupEconomic else 0
    val.append((i,name,iso3a,is_group,is_region))
    
sql = "INSERT INTO economies \
    VALUES (%s,%s,%s,%s,%s);"
cursor.executemany(sql,val)    
db.commit()
print(cursor.rowcount, "Record inserted successfully into economies table")

308 Record inserted successfully into economies table


# Indicator Table

In [29]:
val = []
for i in df[['IndicatorCode','IndicatorCategory','Indicator']].drop_duplicates().values:
    val.append(tuple(i))
for i in df2[['IndicatorCode','IndicatorCategory','Indicator']].drop_duplicates().values:
    val.append(tuple(i))

In [37]:
# Create Table in MySQL
# Indicator(icode, Category, Indicator)
cursor.execute('DROP TABLE IF EXISTS indicators;')
cursor.execute("create TABLE indicators (\
                icode  VarChar(10) UNIQUE NOT NULL,\
                category VarChar(60) NOT NULL,\
                indicator VarChar(100),\
                PRIMARY KEY (icode));")

In [38]:
sql = "INSERT INTO indicators \
    VALUES (%s,%s,%s);"
cursor.executemany(sql,val)    
db.commit()
print(cursor.rowcount, "Record inserted successfully into indicator table")

6 Record inserted successfully into indicator table


# Product Table

In [74]:
val = []
for i in df[['ProductCode','Product','ProductClassificationCode','ProductClassification']].drop_duplicates().values:
    val.append(tuple(i))
for i in df2[['ProductCode','Product','ProductClassificationCode','ProductClassification']].drop_duplicates().values:
    val.append(tuple(i))

In [4]:
# Create Table in MySQL
# Products(pcode, Product, Classification, ClassificationCode)
cursor.execute('DROP TABLE IF EXISTS products;')
cursor.execute("create TABLE products (\
                pcode  VarChar(10) UNIQUE NOT NULL,\
                product VarChar(70) NOT NULL,\
                classification_code VarChar(10) NOT NULL,\
                classification VarChar(70),\
                PRIMARY KEY (pcode));")

In [76]:
sql = "INSERT INTO products \
    VALUES (%s,%s,%s,%s);"
cursor.executemany(sql,val)    
db.commit()
print(cursor.rowcount, "Record inserted successfully into product table")

218 Record inserted successfully into product table


In [65]:
val

[('AG',
  'Agricultural products',
  'SITC3',
  'Merchandise - SITC Revision 3 (aggregates)'),
 ('AGFO', 'Food', 'SITC3', 'Merchandise - SITC Revision 3 (aggregates)'),
 ('MA', 'Manufactures', 'SITC3', 'Merchandise - SITC Revision 3 (aggregates)'),
 ('MACH', 'Chemicals', 'SITC3', 'Merchandise - SITC Revision 3 (aggregates)'),
 ('MACHPH',
  'Pharmaceuticals',
  'SITC3',
  'Merchandise - SITC Revision 3 (aggregates)'),
 ('MACL', 'Clothing', 'SITC3', 'Merchandise - SITC Revision 3 (aggregates)'),
 ('MAIS',
  'Iron and steel',
  'SITC3',
  'Merchandise - SITC Revision 3 (aggregates)'),
 ('MAMT',
  'Machinery and transport equipment',
  'SITC3',
  'Merchandise - SITC Revision 3 (aggregates)'),
 ('MAMTAU',
  'Automotive products',
  'SITC3',
  'Merchandise - SITC Revision 3 (aggregates)'),
 ('MAMTOF',
  'Office and telecom equipment',
  'SITC3',
  'Merchandise - SITC Revision 3 (aggregates)'),
 ('MAMTOTEP',
  'Electronic data processing and office equipment',
  'SITC3',
  'Merchandise - SITC

# Trade Values Table

In [66]:
df.columns

Index(['IndicatorCategory', 'IndicatorCode', 'Indicator', 'ReporterCode',
       'ReporterISO3A', 'Reporter', 'PartnerCode', 'PartnerISO3A', 'Partner',
       'ProductClassificationCode', 'ProductClassification', 'ProductCode',
       'Product', 'PeriodCode', 'Period', 'FrequencyCode', 'Frequency',
       'UnitCode', 'Unit', 'Year', 'ValueFlagCode', 'ValueFlag', 'Value'],
      dtype='object')

In [6]:
# Create Table in MySQL
# TradeValues(Icode, Rcode, Pcode, Procode, Year, Frequency, Unit_Code,\
#  Unit, ValueFlagCode, ValueFlag,tradevalue, growth_rate, percentage)
cursor.execute('DROP TABLE IF EXISTS trade;')
cursor.execute("create TABLE trade (\
                icode  VarChar(10) NOT NULL,\
                rcode  VarChar(5) NOT NULL,\
                pcode  VarChar(5) NOT NULL,\
                procode VarChar(10) NOT NULL,\
                year SMALLINT UNSIGNED NOT NULL CHECK ( 1950 <year <2050 ),\
                frequency VarChar(6) NOT NULL,\
                unitcode VarChar(5) NOT NULL,\
                unit VarChar(20) NOT NULL,\
                valueflagcode VarChar(5),\
                valueflag VarChar(20),\
                tradevalue INT NOT NULL,\
                growthrate FLOAT(8,4),\
                percentage FLOAT(5,5),\
                PRIMARY KEY (icode, rcode, pcode, procode, year),\
                FOREIGN KEY (icode) REFERENCES trade_db.indicators(icode),\
                FOREIGN KEY (rcode) REFERENCES trade_db.economies(ecode),\
                FOREIGN KEY (pcode) REFERENCES trade_db.economies(ecode),\
                FOREIGN KEY (procode) REFERENCES trade_db.products(pcode)),\
                KEY `IDX_Rcode_Pcode_Icode_Procode` (`rcode`,`pcode`,`icode`,`procode`);")

In [22]:
### Use "spark-submit spark_aggregation.py" instead to load csv into mysql
# count = 0
# for i in df[['IndicatorCode', 'ReporterCode', 'PartnerCode','ProductCode', 'Year',\
#     'Frequency','UnitCode', 'Unit', 'ValueFlagCode', 'ValueFlag', 'Value']].values:
#     try:
#         i[1]=str(int(i[1]))
#     except:
#         pass
#     try:
#         i[2]=str(int(i[2]))
#     except:
#         pass
#     i = np.concatenate((i,[None,None]))
#     sql = "INSERT INTO trade\
#     VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
#     if str(i[8]) == 'nan': i[8] = None
#     if str(i[9]) == 'nan': i[9] = None
#     cursor.execute(sql,tuple(i))
#     db.commit()
#     count+=1
# print(count, "Record inserted successfully into trade table")

572501 Record inserted successfully into trade-values table


In [ ]:
# val = []
# count =0
# for i in df2[['IndicatorCode', 'ReporterCode', 'PartnerCode','ProductCode', 'Year',\
#     'Frequency','UnitCode', 'Unit', 'ValueFlagCode', 'ValueFlag', 'Value']].values:
#     try:
#         i[1]=str(int(i[1]))
#     except:
#         pass
#     try:
#         i[2]=str(int(i[2]))
#     except:
#         pass
#     i = np.concatenate((i,[None,None]))
# #     if str(i[8]) == 'nan': i[8] = None
# #     if str(i[9]) == 'nan': i[9] = None
# #     val.append(tuple(i))
# #     count+=1
#     sql = "INSERT INTO trade \
#     VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
#     if str(i[8]) == 'nan': i[8] = None
#     if str(i[9]) == 'nan': i[9] = None
#     try:
#         cursor.execute(sql,tuple(i))
#     except mysql.connector.Error as error:
#         print("Failed to insert into MySQL table {}".format(error))
#         print(i[10])
#         continue
#     db.commit()
#     count+=1
# print(count, "Record inserted successfully into trade table")

In [ ]:
del df2

In [ ]:
df2[df2.Value == -4114]

# Explore Merchandise Trade

In [52]:
ax = df[df.IndicatorCode == 'ITS_MTV_AX']

In [71]:
ax[['ReporterCode','Reporter']].drop_duplicates(),len(ax.ReporterCode.unique()),len(ax.Reporter.unique())

(       ReporterCode                                           Reporter
 98992           970                                               Asia
 99012           928                                European Union (28)
 99015           OWE                              Non-EU western Europe
 99018           OCI                                          Other CIS
 99067           942                                   Pacific Alliance
 ...             ...                                                ...
 561549          580                           Northern Mariana Islands
 561692          LFX                     LDC non-fuel mineral exporters
 562223          981                                       BRIC members
 564358          944  Southern Common Market (MERCOSUR) excluding Ve...
 565840          876                          Wallis and Futuna Islands
 
 [278 rows x 2 columns], 278, 278)

In [63]:
df[['ProductClassificationCode','ProductClassification','ProductCode','Product']].drop_duplicates()

,ProductClassificationCode,ProductClassification,ProductCode,Product
0,SITC3,Merchandise - SITC Revision 3 (aggregates),AG,Agricultural products
5882,SITC3,Merchandise - SITC Revision 3 (aggregates),AGFO,Food
11731,SITC3,Merchandise - SITC Revision 3 (aggregates),MA,Manufactures
17669,SITC3,Merchandise - SITC Revision 3 (aggregates),MACH,Chemicals
22294,SITC3,Merchandise - SITC Revision 3 (aggregates),MACHPH,Pharmaceuticals
26181,SITC3,Merchandise - SITC Revision 3 (aggregates),MACL,Clothing
32072,SITC3,Merchandise - SITC Revision 3 (aggregates),MAIS,Iron and steel
36757,SITC3,Merchandise - SITC Revision 3 (aggregates),MAMT,Machinery and transport equipment
41388,SITC3,Merchandise - SITC Revision 3 (aggregates),MAMTAU,Automotive products
45990,SITC3,Merchandise - SITC Revision 3 (aggregates),MAMTOF,Office and telecom equipment


In [64]:
len(df[['ProductClassificationCode','ProductClassification','ProductCode','Product']].drop_duplicates())

31

In [79]:
c = ax.ReporterCode.unique()
d = ax.PartnerCode.unique()
len(set(list(c)+list(d)))

290

In [73]:
a = ax.Reporter.unique()
b = ax.Partner.unique()
for i in b:
    if i not in a:
        print(i)

Extra Andean Community Trade
Extra CEMAC Trade
Extra MERCOSUR Trade
Extra NAFTA Trade
Extra ASEAN Trade
Extra Pacific Alliance Trade
Extra ECOWAS Trade
Extra SADC Trade
Extra EU(28) Trade
Extra COMESA Trade
Extra WAEMU Trade
Extra EU Trade


In [36]:
df[(df.ProductCode=='TO') & (df.Partner!= 'World')][['Reporter','Partner']].drop_duplicates()

,Reporter,Partner
86545,European Union,European Union
98972,European Union,Extra EU Trade
545527,South and Central America and the Caribbean,"Commonwealth of Independent States (CIS), incl..."
545535,North America,Non-EU western Europe
545538,South and Central America and the Caribbean,Australia and New Zealand
...,...,...
571785,Asia,United States of America
571869,Other Asia,European Union (28)
571892,Six East Asian traders,Mexico
571944,Asia,South and Central America and the Caribbean


In [62]:
df.PartnerISO3A.unique(),\
df.ReporterISO3A.unique(),\
len(df.ReporterISO3A.unique()),\
len(df.PartnerCode.unique()),\
len(df.Reporter.unique()),\
len(df.ReporterCode.unique())

(array([nan, 'EEC', 'JPN', 'CHN', 'ZAF', 'BRA', 'IND', 'RUS', 'USA', 'MEX'],
       dtype=object),
 array(['KEN', 'AUS', 'MAC', 'IRL', 'BGD', 'FJI', 'LVA', 'KOR', 'JPN',
        'KWT', 'AUT', 'ARG', 'FIN', 'BHS', 'CZE', nan, 'MUS', 'HTI', 'COG',
        'CYM', 'MRT', 'BRN', 'BRB', 'JAM', 'ARM', 'MLT', 'MLI', 'BEN',
        'MEX', 'JOR', 'GAB', 'ALB', 'LTU', 'GRL', 'ECU', 'KAZ', 'AZE',
        'LBY', 'IND', 'HUN', 'PYF', 'HND', 'GUY', 'EST', 'ERI', 'GIN',
        'DZA', 'DDR', 'GRD', 'LBN', 'GTM', 'BEL', 'BLX', 'BMU', 'BHR',
        'DNK', 'BOL', 'BTN', 'ETH', 'MWI', 'MDG', 'DMA', 'BWA', 'BLR',
        'MDA', 'OMN', 'LAO', 'MSR', 'ANT', 'ABW', 'MNG', 'NLD', 'DOM',
        'SLV', 'CUB', 'CHL', 'GEO', 'LUX', 'NPL', 'NZL', 'PAN', 'GMB',
        'NOR', 'BRA', 'FSM', 'PRY', 'NGA', 'NIC', 'NER', 'POL', 'VUT',
        'PRT', 'MYS', 'MDV', 'PNG', 'CIV', 'PAK', 'SYC', 'RUS', 'LBR',
        'PER', 'QAT', 'NCL', 'SAU', 'CHN', 'KNA', 'RWA', 'DEU', 'PHL',
        'PLW', 'VCT', 'STP', 'ZWE', 'SVN', '

# Explore SERVICES Table

In [1]:
file2= r"C:\Users\Lin\Desktop\551 Foundations of Data Management\Dataset\services_annual_dataset\services_annual_dataset.csv"

In [5]:
df2 = pd.read_csv(file2,sep=',',encoding='cp1252') 

C:\Users\Lin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,6,7,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df2[['IndicatorCategory','IndicatorCode','Indicator']].drop_duplicates()

,IndicatorCategory,IndicatorCode,Indicator
0,Trade in commercial services,ITS_CS_AM5,Commercial services imports by sector - annual (1980-2013)
78052,Trade in commercial services,ITS_CS_AM6,Commercial services imports by sector and partner - annual (2005-2019)
1771216,Trade in commercial services,ITS_CS_AX5,Commercial services exports by sector - annual (1980-2013)
1850079,Trade in commercial services,ITS_CS_AX6,Commercial services exports by sector and partner - annual (2005-2019)


In [83]:
for i in df2.Reporter.unique():
    if i not in df.Reporter.unique():
        print(i)

Côte d'Ivoire
Sao Tomé and Principe
Curaçao
South Sudan


In [111]:
for i in df2.ReporterCode.unique():
    try:
        i = int(i)
    except:
        continue
    if str(i) not in country_dic.keys():
        print(str(i))

728
728


In [8]:
serv_country_R_dic = {}
serv_country_P_dic = {}
for i in df2[['ReporterCode','Reporter','ReporterISO3A']].drop_duplicates().values:
    try:
        i[0] = int(i[0])
    except:
        pass
    if str(i[0]) not in serv_country_R_dic.keys():
        serv_country_R_dic[str(i[0])]=i[1:]
for i in df2[['PartnerCode','Partner','PartnerISO3A']].drop_duplicates().values:
    try:
        i[0] = int(i[0])
    except:
        pass
    if str(i[0]) not in serv_country_P_dic.keys():
        serv_country_P_dic[str(i[0])]=i[1:]